# Lab Assignment 1: Rock, Paper, Scissors

## Write and Run Your Own Code to Implement a Rock-Paper-Scissors Model

In [2]:
import numpy as np
import random
# np.random.seed(0)
# random.seed(0)


def transfer_function(a):
   return np.tanh(-a)


def transfer_derivative(a):
   return -(1 - np.tanh(-a)**2)


def loss_function(predicted, target):#Using MSE as going with Cross Entropy changed entirety of code in frontprop and we had written already
   return 0.5 * np.mean((predicted - target)**2)


class RNN:
   #Before we just called this c lass the neuron
   def __init__(self, input_size, hidden_size, output_size, lr=1e-4):
       self.input_size = input_size
       self.hidden_size = hidden_size
       self.output_size = output_size
       self.lr = lr #learning rate, we set that in arg


       #x stands for input and y for otput in gen
       #randomizing these w out the fixed seeds as I saw too much repetition when i ran the original
       self.weight_xh = np.random.randn(hidden_size, input_size) * 0.01 #Weight (hidden_size x input_size)   [input-to-hidden weights]
       self.weight_hh = np.random.randn(hidden_size, hidden_size) * 0.01 #Weight hidden to hidden, basically recurrent weight
       self.bias_h  = np.zeros(hidden_size) #


       self.weight_hy = np.random.randn(output_size, hidden_size) * 0.01
       self.bias_y  = np.zeros(output_size)


   def forward_propagate(self, inputs):
       T = inputs.shape[0] # T is length of list
       hidden_states = []
       outputs = []


       hidden_state_prev = np.zeros(self.hidden_size)


       for t in range(T):
           input_t = inputs[t]
           activation_t = (self.weight_xh @ input_t) + (self.weight_hh @ hidden_state_prev) + self.bias_h
           hidden_t = transfer_function(activation_t) #hidden states
           out_t = (self.weight_hy @ hidden_t) + self.bias_y


           hidden_states.append(hidden_t)
           outputs.append(out_t)
           hidden_state_prev = hidden_t


       return hidden_states, outputs


   def back_propagate(self, inputs, hidden_states, outputs, targets):
       T = inputs.shape[0] #timesteps


       #all these are to get the gradients from each, let dW = derivative of weight
       #same size shape as weight from forwardprop
       dW_xh = np.zeros_like(self.weight_xh)
       dW_hh = np.zeros_like(self.weight_hh)
       db_h  = np.zeros_like(self.bias_h)
       dW_hy = np.zeros_like(self.weight_hy)
       db_y  = np.zeros_like(self.bias_y)


       dh_next = np.zeros(self.hidden_size)  #gradient hidden state from subsequent
       total_loss = 0.0


       for t in reversed(range(T)):
           y_t = outputs[t] #use output variables from engineering lingo for variable clarification
           h_t = hidden_states[t]
           x_t = inputs[t]
           if t == 0:
               h_prev = np.zeros(self.hidden_size) #work out the zero case like no previous hidden state if 0s
           else:
               h_prev = hidden_states[t - 1]


           # MSE loss (instead of cross entropy) at each timestep
           total_loss += loss_function(y_t, targets[t])
           dy = (y_t - targets[t])  # partial deriv
           dW_hy += np.outer(dy, h_t) #back to gradient weight from hidden to output y
           db_y  += dy
           dh = (self.weight_hy.T @ dy) + dh_next #passing error to hidden layer, use the @ thingy for matrix mult
           da_t = dh * (-(1.0 - (h_t**2)))
           dW_xh += np.outer(da_t, x_t) #get grad wrt input to hidden & hiddenxhidden
           dW_hh += np.outer(da_t, h_prev)
           db_h  += da_t #grad to hid biass
           dh_next = self.weight_hh.T @ da_t #grad passed further backwards lol hence function


       #this is gradient descent all at once, had to do this on paper, not sure if it works lol
       self.weight_xh -= self.lr * dW_xh
       self.weight_hh -= self.lr * dW_hh
       self.bias_h  -= self.lr * db_h
       self.weight_hy -= self.lr * dW_hy
       self.bias_y  -= self.lr * db_y
       return total_loss


#in these args, dim is diff for each arg (i think?) at diff points
def network_init(input_dim, hidden_dim, output_dim, lr=1e-4): #simple func initialize RNN class above, wasn;t orig like this
   return RNN(input_dim, hidden_dim, output_dim, lr) #easier to work w/




def rock_paper_scissor(history, num_epochs=50): #now we can use our history inputs and run through the RNN and plug into this
   print("Initializing RNN for Rock-Paper-Scissors...")
   input_dim = 6  #3 1-hot for each human nd cpu, respectively = 6
   hidden_dim = 50 #50 hiddn neurons for now
   output_dim = 3  #obviously (3 options for y)
   learning_rate = 1e-3
   rnn = network_init(input_dim, hidden_dim, output_dim, lr=learning_rate) #object
   steps = len(history)
   store_inputs = []
   store_comp_input = []
   print(f"Collecting {steps} steps from 'history'...") #user can check if all of data being used


   for i in range(steps):
       human_move = history[i, 0]
       computer_move = history[i, 1]
       inp_vec = np.zeros(input_dim) #debugged this forever before realizing needed np here
       inp_vec[human_move] = 1
       inp_vec[computer_move + 3] = 1
       store_inputs.append(inp_vec)
       comp_vec = np.zeros(output_dim) #1-hot for cpu move of shape (3,)\
       """
       Logic if cpu move is 0 we get vector [1,0,0] and so on [0,1,0] or [0,0,1]
       """
       comp_vec[computer_move] = 1
       store_comp_input.append(comp_vec)


   # Build wincond is the target we built originally and it starts off empty and builds through the network
   print("Building 'wincond' targets from 'store_comp_input'...")
   wincond = []
   for x in store_comp_input:
       if np.array_equal(x, [1, 0, 0]): #cpu rock
           wincond.append(np.array([ 0,  1, -1]))
       elif np.array_equal(x, [0, 1, 0]): #cpu paper
           wincond.append(np.array([-1,  0,  1]))
       else:  #cpu scissors
           wincond.append(np.array([ 1, -1,  0]))


   store_inputs = np.array(store_inputs) #TYPING!!!! (steps,6)
   wincond = np.array(wincond) #(steps,3)


   # Training loop
   for epoch in range(num_epochs):
       hidden_states, outputs = rnn.forward_propagate(store_inputs) #forward pass
       total_loss = rnn.back_propagate(store_inputs, hidden_states, outputs, wincond) #backward paass WITH wincond targets
       print(f"Epoch {epoch+1}/{num_epochs}, MSE Loss = {total_loss:.4f}")


   #need one more forward pass
   _, outputs_after = rnn.forward_propagate(store_inputs)
   final_output = outputs_after[-1] #last timestep
   recommended_max_dim = np.argmax(final_output) #largest dimension
   moves_map = {0: "Rock", 1: "Paper", 2: "Scissors"}
   recommended_move = moves_map[recommended_max_dim]
   print(f"\nAlgorithm recommends: {recommended_move}\n")
   return recommended_move


#executing the whole thang!
history = np.loadtxt("./training.txt", dtype=int).reshape(-1, 3)
recommended = rock_paper_scissor(history, num_epochs=50) #arbitrarily decided on 50
print("Recommended move after training loop:", recommended)

Initializing RNN for Rock-Paper-Scissors...
Building 'wincond' targets from 'store_comp_input'...
Epoch 1/50, MSE Loss = 112.0114
Epoch 2/50, MSE Loss = 110.1302
Epoch 3/50, MSE Loss = 109.1409
Epoch 4/50, MSE Loss = 108.4761
Epoch 5/50, MSE Loss = 107.8566
Epoch 6/50, MSE Loss = 107.1138
Epoch 7/50, MSE Loss = 106.1078
Epoch 8/50, MSE Loss = 104.6854
Epoch 9/50, MSE Loss = 102.6540
Epoch 10/50, MSE Loss = 99.7673
Epoch 11/50, MSE Loss = 95.7250
Epoch 12/50, MSE Loss = 90.2045
Epoch 13/50, MSE Loss = 82.9519
Epoch 14/50, MSE Loss = 73.9579
Epoch 15/50, MSE Loss = 63.6941
Epoch 16/50, MSE Loss = 53.2310
Epoch 17/50, MSE Loss = 43.8863
Epoch 18/50, MSE Loss = 36.3413
Epoch 19/50, MSE Loss = 30.1899
Epoch 20/50, MSE Loss = 24.6889
Epoch 21/50, MSE Loss = 19.5237
Epoch 22/50, MSE Loss = 14.7890
Epoch 23/50, MSE Loss = 10.7402
Epoch 24/50, MSE Loss = 7.5903
Epoch 25/50, MSE Loss = 5.3619
Epoch 26/50, MSE Loss = 3.8751
Epoch 27/50, MSE Loss = 2.8780
Epoch 28/50, MSE Loss = 2.1744
Epoch 29/50

## What to Submit?
1. A link to your repository (remember to add your Lab TA as a collaborator to this repository, or they won't be able to grade your work!).
2. Your training data in training.txt.
3. Your model evaluation data.
4. A brief write-up describing your algorithm in a couple of sentences, including figures if necessary (your design on paper).
5. Finally, please individually briefly reflect on your experience in a couple of sentences. What are you taking away from this lab? 